In [6]:
import csv
import pandas as pd
import numpy as np
import nvector as nv

import folium
import folium.plugins as plugins
import ipywidgets as widgets
from IPython.display import HTML
from IPython.display import display

#parse and load csv file into a Pandas DataFrame
csvfile = 'https://raw.githubusercontent.com/rkalyanapurdue/smolensk/master/Division.csv'
df = pd.read_csv(csvfile,parse_dates=['MAP_DATE'])

#group the data by Army group and num
grouped_div = df.groupby(['Army_Group','Num_Name'])

def get_div_origin_dest(div_data_df):
    div_data = dict()
    for row in div_data_df.itertuples():
        if row.MAP_DATE not in div_data:
            div_data[row.MAP_DATE] = dict()
            div_data[row.MAP_DATE]['POINT_Y'] = row.POINT_Y
            div_data[row.MAP_DATE]['POINT_X'] = row.POINT_X
    div_locs = []
    for key in sorted(div_data):
        div_loc = dict()
        div_loc['POINT_Y'] = div_data[key]['POINT_Y']
        div_loc['POINT_X'] = div_data[key]['POINT_X']
        div_locs.append(div_loc)
    #there is an origin and dest
    if len(div_locs) > 1:
        origin = div_locs[0]
        dest = div_locs[len(div_locs)-1]
        return [[origin['POINT_Y'],origin['POINT_X']],
                [dest['POINT_Y'],dest['POINT_X']]]
    else:
        return None

origin_lats = []
origin_lons = []
dest_lats = []
dest_lons = []

#iterate over divisions
for group_key in grouped_div.groups.keys():
    #weed out groups that don't have a valid num
    if isinstance(group_key[1],str):
        div_df = grouped_div.get_group(group_key)[['POINT_X','POINT_Y','MAP_DATE']]
        #get division's origin and destination
        div_coords = get_div_origin_dest(div_df)
        if div_coords is not None:
            origin_lats.append(div_coords[0][0])
            origin_lons.append(div_coords[0][1])
            dest_lats.append(div_coords[1][0])
            dest_lons.append(div_coords[1][1])

points = nv.GeoPoint(latitude=origin_lats,longitude=origin_lons,degrees=True)
nvectors = points.to_nvector()
n_EM_E = nvectors.mean_horizontal_position()
g_EM_E = n_EM_E.to_geo_point()
lat, lon = g_EM_E.latitude_deg, g_EM_E.longitude_deg
mean_origin = [lat[0],lon[0]]

points = nv.GeoPoint(latitude=dest_lats,longitude=dest_lons,degrees=True)
nvectors = points.to_nvector()
n_EM_E = nvectors.mean_horizontal_position()
g_EM_E = n_EM_E.to_geo_point()
lat, lon = g_EM_E.latitude_deg, g_EM_E.longitude_deg
mean_dest = [lat[0],lon[0]]

#now create the map
div_map = folium.Map([54.78, 32.04],zoom_start=6)

#first the markers
folium.CircleMarker(mean_origin,radius=4,color='green').add_to(div_map)
folium.CircleMarker(mean_dest,radius=4,color='red').add_to(div_map)
orig_dest_pair = [mean_origin,mean_dest]
vect_line = folium.PolyLine(locations=orig_dest_pair,weight=1,color='black')
div_map.add_child(vect_line)
    
div_map